In [22]:
import numpy as np
import torch.nn.functional as F
import torch
import torch.nn as nn
from torchsummary import summary
import matplotlib.pyplot as plt
from math import floor
from torch.utils import data
import math
from datasets.datasets import RobotNavDataset
from utils.logging import setup_logger

plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
# for now just generate some random data, don't have access to kaggle at work
CLASSES = ['fine_concrete', 'concrete', 'soft_tiles', 'tiled', 'soft_pvc',
           'hard_tiles_large_space', 'carpet', 'hard_tiles', 'wood']

train_set = RobotNavDataset('data/')

In [29]:
class ShallowCNN(nn.Module):
    def __init__(self, 
                 sequence_length=128,
                 n_channels=20,
                 conv1_width = 10,
                 max_pool_kernel_size=25,
                 max_pool_stride=5):
        '''
        sequence_length: number of measurements in each sequennce
        n_channles: number of channels for convolution layers
        max_pool_kernel_size: size along 2nd axis (not sure on this wording)
        max_pool_stride: stride along 2nd axis
        '''
        super(ShallowCNN, self).__init__()
        self.n_channels = n_channels
        
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_channels, kernel_size=(1,conv1_width))
        self.conv2 = nn.Conv2d(in_channels=n_channels, out_channels=n_channels, kernel_size=(10,1))
        self.post_conv_width = math.floor(1 + (sequence_length - conv1_width)/self.conv1.stride[1])
        
        self.conv2_bn = nn.BatchNorm2d(n_channels)
        self.max_pool = nn.MaxPool2d(kernel_size=(1, max_pool_kernel_size), stride=(1, max_pool_stride))
        max_pool_w_out = math.floor(1 + (self.post_conv_width - max_pool_kernel_size)/max_pool_stride)
        
        self.linear = nn.Linear(in_features=n_channels*max_pool_w_out, out_features=9)
        
    def forward(self, x):
        x = self.conv1(x)
        #x = F.dropout2d(x, p=0.5)
        
        x = self.conv2(x)
        x = F.dropout2d(x, p=0.3, training=self.training)
        x = F.elu(self.conv2_bn(x))
        #x = F.relu(self.conv2_bn(x))
        
        x = x.view((-1, self.n_channels, self.post_conv_width))
        x = self.max_pool(x)
        
        x = x.view((x.size(0), -1))
        x = self.linear(x)
        
        x = F.log_softmax(x, dim=1)
        return x

In [71]:
'''
class ShallowCNN(nn.Module):
    def __init__(self, n_conv_channels=40, 
                 n_signal_channels=10, 
                 pool_size=25, 
                 pool_stride=15,
                 n_out=10):
        
        super(ShallowCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=n_conv_channels, 
                               kernel_size=(1,n_signal_channels))
        
        self.conv2 = nn.Conv2d(in_channels=n_conv_channels, 
                               out_channels=n_conv_channels, 
                               kernel_size=(n_signal_channels,1))
        
        self.conv2_bn = nn.BatchNorm2d(n_signal_channels)
        
        self.max_pool = nn.MaxPool2d(kernel_size=(1,pool_size), stride=(1,pool_stride))
        # Need to determine how to calculate 7
        self.linear = nn.Linear(in_features=40*7, out_features=n_out)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = F.elu(self.conv2_bn(x))
        # need to determine how to calculate this size
        x = x.view((-1,40,119))
        x = self.max_pool(x)
        x = x.view((x.size(0), -1))
        x = self.linear(x)
        return x
'''

'\nclass ShallowCNN(nn.Module):\n    def __init__(self, n_conv_channels=40, \n                 n_signal_channels=10, \n                 pool_size=25, \n                 pool_stride=15,\n                 n_out=10):\n        \n        super(ShallowCNN, self).__init__()\n        \n        self.conv1 = nn.Conv2d(in_channels=1, \n                               out_channels=n_conv_channels, \n                               kernel_size=(1,n_signal_channels))\n        \n        self.conv2 = nn.Conv2d(in_channels=n_conv_channels, \n                               out_channels=n_conv_channels, \n                               kernel_size=(n_signal_channels,1))\n        \n        self.conv2_bn = nn.BatchNorm2d(n_signal_channels)\n        \n        self.max_pool = nn.MaxPool2d(kernel_size=(1,pool_size), stride=(1,pool_stride))\n        # Need to determine how to calculate 7\n        self.linear = nn.Linear(in_features=40*7, out_features=n_out)\n        \n    def forward(self, x):\n        x = self.

In [30]:
model = ShallowCNN()
#model.double()
model.to('cpu')
# foo.cuda()

ShallowCNN(
  (conv1): Conv2d(1, 20, kernel_size=(1, 10), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(10, 1), stride=(1, 1))
  (conv2_bn): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool): MaxPool2d(kernel_size=(1, 25), stride=(1, 5), padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=380, out_features=9, bias=True)
)

In [31]:
summary(model, input_size=train_set.train[0].shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 20, 10, 119]             220
            Conv2d-2           [-1, 20, 1, 119]           4,020
       BatchNorm2d-3           [-1, 20, 1, 119]              40
         MaxPool2d-4               [-1, 20, 19]               0
            Linear-5                    [-1, 9]           3,429
Total params: 7,709
Trainable params: 7,709
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.22
Params size (MB): 0.03
Estimated Total Size (MB): 0.26
----------------------------------------------------------------


## Create data loaders

In [32]:
train_subset[0][0].shape

torch.Size([1, 10, 128])

In [33]:
train_size = floor(0.8*len(train_set))
test_size = floor(0.2*len(train_set))

train_subset, test_subset = data.random_split(train_set, (train_size, test_size))

train_loader = torch.utils.data.DataLoader(train_subset,
                                          batch_size=64,
                                          shuffle=True)

# Don't think we actually need shuffle here...
test_loader = torch.utils.data.DataLoader(test_subset,
                                         batch_size=64,
                                         shuffle=True)

## Optimizer

In [34]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Train

In [35]:
from utils.training import train, test
logger = setup_logger(logfile='foo.log', console_out=True)

In [36]:
n_epochs = 10
log_interval = 40
model.double()

loss_func = F.nll_loss

for epoch in range(1, n_epochs+1):
    train(model, train_loader, optimizer, loss_func, epoch, log_interval=log_interval, log_func=logger.info)
    test(model, test_loader, loss_func, log_func=logger.info)

Train Epoch: 1	Loss: 2.385076
Train Epoch: 1	Loss: 2.385076
Train Epoch: 1	Loss: 2.006884
Train Epoch: 1	Loss: 2.006884

Test set: Average loss: 1.9891, Accuracy: 153/762 (20%)


Test set: Average loss: 1.9891, Accuracy: 153/762 (20%)

Train Epoch: 2	Loss: 1.986988
Train Epoch: 2	Loss: 1.986988
Train Epoch: 2	Loss: 1.984002
Train Epoch: 2	Loss: 1.984002

Test set: Average loss: 1.9239, Accuracy: 185/762 (24%)


Test set: Average loss: 1.9239, Accuracy: 185/762 (24%)

Train Epoch: 3	Loss: 1.931040
Train Epoch: 3	Loss: 1.931040
Train Epoch: 3	Loss: 1.913984
Train Epoch: 3	Loss: 1.913984

Test set: Average loss: 1.8318, Accuracy: 223/762 (29%)


Test set: Average loss: 1.8318, Accuracy: 223/762 (29%)

Train Epoch: 4	Loss: 1.748256
Train Epoch: 4	Loss: 1.748256


KeyboardInterrupt: 

In [13]:
logger.info('asdfasd')

asdfasd
asdfasd
asdfasd
asdfasd


In [105]:
n_epochs = 40
log_interval = 80
model.train()
model.double()
for epoch in range(1, n_epochs+1):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))

Train Epoch: 1 [0/3810 (0%)]	Loss: 1.347208
Train Epoch: 2 [0/3810 (0%)]	Loss: 1.364793
Train Epoch: 3 [0/3810 (0%)]	Loss: 1.187895
Train Epoch: 4 [0/3810 (0%)]	Loss: 1.231539
Train Epoch: 5 [0/3810 (0%)]	Loss: 1.227063
Train Epoch: 6 [0/3810 (0%)]	Loss: 1.244717
Train Epoch: 7 [0/3810 (0%)]	Loss: 1.368650
Train Epoch: 8 [0/3810 (0%)]	Loss: 1.259439
Train Epoch: 9 [0/3810 (0%)]	Loss: 1.581491
Train Epoch: 10 [0/3810 (0%)]	Loss: 1.418440
Train Epoch: 11 [0/3810 (0%)]	Loss: 1.455553
Train Epoch: 12 [0/3810 (0%)]	Loss: 1.388214
Train Epoch: 13 [0/3810 (0%)]	Loss: 1.464187
Train Epoch: 14 [0/3810 (0%)]	Loss: 1.320769
Train Epoch: 15 [0/3810 (0%)]	Loss: 1.313512
Train Epoch: 16 [0/3810 (0%)]	Loss: 1.289399
Train Epoch: 17 [0/3810 (0%)]	Loss: 1.568147
Train Epoch: 18 [0/3810 (0%)]	Loss: 1.457238
Train Epoch: 19 [0/3810 (0%)]	Loss: 1.330974
Train Epoch: 20 [0/3810 (0%)]	Loss: 1.239509
Train Epoch: 21 [0/3810 (0%)]	Loss: 1.078833
Train Epoch: 22 [0/3810 (0%)]	Loss: 1.168862
Train Epoch: 23 [0/

In [106]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in train_loader:
        #data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(train_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(train_loader.dataset),
100. * correct / len(train_loader.dataset)))


Test set: Average loss: 1.1127, Accuracy: 2395/3810 (63%)



In [17]:
model

ShallowCNN(
  (conv1): Conv2d(1, 40, kernel_size=(1, 10), stride=(1, 1))
  (conv2): Conv2d(40, 40, kernel_size=(10, 1), stride=(1, 1))
  (conv2_bn): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool): MaxPool2d(kernel_size=(1, 25), stride=(1, 5), padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=760, out_features=10, bias=True)
)